In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
#key imports

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [ ]:
#processing images
import cv2 as cv
import copy
import numpy as np
import os

num = 1000

cons = ""
sub = ""

working_directory = "/content/drive/MyDrive/Predict"


blank = np.zeros((1080, 1920, 1), dtype=np.uint8)

class contourFinder:
    def __init__(self) -> None:
        cnt = 0     
        self.__images = self.__imageReader()
        for image in self.__images:
          self.currimg = image
          try:
            cnt += 1
            dir = "/content/drive/MyDrive/Predict/processed-" + str(cnt) + ".png"
            self.__preProcessing(image)
            self.findContours(100, 300)
            cv.imwrite(dir, self.clone)
          except ZeroDivisionError:
            print("error")
            continue
          except KeyboardInterrupt:
            break

    def __imageReader(self, folder = working_directory):
        images = []
        for filename in os.listdir(folder):
          img = cv.imread(os.path.join(folder, filename))
          if img is not None:
            images.append(img)
        return images


    def __preProcessing(self, image):
        self.__gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        self.__blurred = cv.GaussianBlur(self.__gray, (5, 5), 1)

    def __sharpenImage(self):
        __kernel = np.array([[-1,-1,-1],
                   [-1, 9,-1],
                   [-1,-1,-1]])
        self.__sharpened = cv.filter2D(self.__image, -1, __kernel)

        return self.__sharpened

    def findContours(self, t1, t2):
        tmp = self.__blurred

        self.__canneyed = cv.Canny(tmp, t1, t2)
        (self.cnts, _) = cv.findContours(self.__canneyed, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
        self.clone = copy.deepcopy(blank)

        for c in self.cnts:
            M = cv.moments(c)

            if M["m00"] != 0:
              cX, cY = (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"]))
            else: cX, cY = 0, 0

            cv.circle(self.clone, (cX, cY), 3, (255, 255, 255), -1)

if __name__ == "__main__":
    image = contourFinder()

In [ ]:
#creating custom dataset from directory

h, w = 1080, 1920

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Train",
    labels = "inferred",
    label_mode = "int",
    color_mode = "grayscale",
    batch_size = 20,
    image_size = (h, w),
    shuffle = True,
    seed = 123,
    validation_split = 0.1,
    subset = "training"
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Train",
    labels = "inferred",
    label_mode = "int",
    color_mode = "grayscale",
    batch_size = 20,
    image_size = (h, w),
    shuffle = True,
    seed = 123,
    validation_split = 0.1,
    subset = "validation"
)

Found 240 files belonging to 12 classes.
Using 216 files for training.
Found 240 files belonging to 12 classes.
Using 24 files for validation.


In [ ]:
#preprocessing and classifier
IMG_SIZE = 600 # All images will be resized to 800x800

def format_example(image, label):
  """
  returns an image that is reshaped to IMG_SIZE
  """
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

def normalize(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image, label

ds_train = ds_train.map(format_example)
ds_validation = ds_validation.map(format_example)

#calssifier
classname = ["Aquarius", "Aries", "Cancer", "Capricorn", "Gemini", "Leo", "Libra", "Pisces", "Sagittarius", "Scorpio", "Taurus", "Virgo"]

In [ ]:
#convolutional layers
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(600, 600, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

In [ ]:
#dense neural network
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(12))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 598, 598, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 299, 299, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 297, 297, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 148, 148, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 146, 146, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 73, 73, 64)       0

In [ ]:
#training model
model.compile(optimizer='adam',
       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
       metrics=['accuracy'])

history = model.fit(ds_train, epochs=3, validation_data=(ds_validation))

Epoch 1/3
11/11 [==============================] - 12s 486ms/step - loss: 6.8808 - accuracy: 0.1065 - val_loss: 2.3875 - val_accuracy: 0.3333
Epoch 2/3
11/11 [==============================] - 4s 266ms/step - loss: 2.0014 - accuracy: 0.3889 - val_loss: 1.1162 - val_accuracy: 0.6667
Epoch 3/3
11/11 [==============================] - 4s 266ms/step - loss: 0.6494 - accuracy: 0.8148 - val_loss: 0.6928 - val_accuracy: 0.7917


In [ ]:
#evaluating model
test_loss, test_acc = model.evaluate(ds_validation, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.6928 - accuracy: 0.7917 - 440ms/epoch - 220ms/step
0.7916666865348816


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def imageReader(folder):
  images = []
  for filename in os.listdir(folder):
    img = cv.imread(os.path.join(folder, filename))
    if img is not None:
      images.append(img)
  return images

image = imageReader(folder="/content/drive/MyDrive/Predict")

for e in image:
  e = (e/127.5) - 1
image = tf.image.resize(image, (600, 600))

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])     

gray = rgb2gray(image) 


predictions = model.predict([gray])
tmp = predictions[0]
print(tmp)
index = 0
for i in range(len(tmp)):
  max = 0
  if tmp[i] > max:
    index = i
print(classname[i])

1/1 [==============================] - 0s 21ms/step
[-1000.02466   169.16464  3354.679    1397.5154   -252.59311  1262.6085
  1607.1628   2091.1477    868.262   -1215.1125    253.0615   2476.9114 ]
Virgo


In [ ]:
predictions = model.predict(ds_validation)
predictions[0]

2/2 [==============================] - 0s 96ms/step


array([-6.5663805 , -0.01828386, -2.1347504 , -2.6848733 ,  9.687871  ,
        3.800414  ,  2.8288617 , -0.9208708 ,  0.68489784,  2.5550249 ,
        3.0461142 ,  0.24904545], dtype=float32)